# Introduction/Project Overview:

In this notebook, I will go over the World Health Organization Life Expectancy dataset. I found a version on kaggle that had been updated and had a higher usability score so I chose to use that. It is linked here if you want to read more about it. [dataset](https://www.kaggle.com/datasets/lashagoch/life-expectancy-who-updated). We are given data on a country, population, health issues, life expectancy, etc. Our goal would be to create a model that can accurately determine the life expectency given a some characteristics on a population. Throughout this notebook I will visualize the data, explain some data preprocessing techniques, construct and evaluate models and analyze the results.

### Data Exploration & Preprocessing:
I will go over the dataset, analyzing its various features, checking for missing values, and gaining insights into the distribution of variables. Prior to building the models, I will preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features to ensure good model performance.

### Model Building & Evaluation:
In this notebook I will try implement a couple of models to try and see which ones are more accurate at predicting life expectancy. This is a supervised learning task since we are given the life expectancy of these population. Additionally this is a regression tasks because we are trying to predict a numerical value rather then put something into a category. For this notebook the models I chose is multiple linear regression, polynomial regression and a neural network. Lastly I will interpret the results of each model. 


### Conclusion:
Finally, I will be discussing potential areas for model improvement, what stood out to me and what were some challanges. The conclusion serves more as a reflection for me on my time working on this notebook. This will serve as a good test for me to keep learning and testing my skills. Lets get to it!

In [27]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Exploration & Preprocessing:
As mentioned earlier I got the dataset from kaggle. The link to that can be found above in the project overview. The download came with a csv file. Since I have it locally on my computer I can eassily access the data as shown below. Some of the first steps we will do before creating a model is to see what our data looks like.

In [28]:
data = pd.read_csv('./data.csv')

Lets take a look at the first couple of entries in our dataset and what columns are int it.

In [29]:
data.head(5)

,Country,Region,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,...,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Life_expectancy
0,Turkiye,Middle East,2015,11.1,13.0,105.8240,1.32,97,65,27.8,...,97,0.08,11006,78.53,4.9,4.8,7.8,0,1,76.5
1,Spain,European Union,2015,2.7,3.3,57.9025,10.35,97,94,26.0,...,97,0.09,25742,46.44,0.6,0.5,9.7,1,0,82.8
2,India,Asia,2007,51.5,67.9,201.0765,1.57,60,35,21.2,...,64,0.13,1076,1183.21,27.1,28.0,5.0,0,1,65.4
3,Guyana,South America,2006,32.8,40.5,222.1965,5.68,93,74,25.3,...,93,0.79,4146,0.75,5.7,5.5,7.9,0,1,67.0
4,Costa Rica,Central America and Caribbean,2006,9.8,11.2,95.2200,4.19,88,86,26.4,...,89,0.16,9110,4.35,2.0,1.9,7.9,0,1,78.2


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2848 entries, 0 to 2847
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Country                      2848 non-null   object 
 1   Region                       2848 non-null   object 
 2   Year                         2848 non-null   int64  
 3   Infant_deaths                2848 non-null   float64
 4   Under_five_deaths            2848 non-null   float64
 5   Adult_mortality              2848 non-null   float64
 6   Alcohol_consumption          2848 non-null   float64
 7   Hepatitis_B                  2848 non-null   int64  
 8   Measles                      2848 non-null   int64  
 9   BMI                          2848 non-null   float64
 10  Polio                        2848 non-null   int64  
 11  Diphtheria                   2848 non-null   int64  
 12  Incidents_HIV                2848 non-null   float64
 13  GDP_per_capita    

So we have 22 columns in total. We have three categorical columns and 19 numeric columns. We also have quite some missing values so we will try and fill them later. The column names are pretty straight forward but if you would like to read more on them there are explanations on kaggle.

Lets see what other information we can get at a glance from our data. 

In [31]:
data.describe()

,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Life_expectancy
count,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.00000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000,2848.000000
mean,2007.500000,30.510955,43.150456,192.962955,4.833032,84.21875,77.250351,25.023455,86.456110,86.227879,0.898862,11430.386938,36.840976,4.886763,4.921173,7.604951,0.202247,0.797753,68.789010
std,4.610582,27.545025,44.604702,114.837818,3.989790,16.00973,18.667562,2.196260,15.111162,15.566384,2.387287,16916.391089,136.851020,4.441881,4.528915,3.159514,0.401746,0.401746,9.388885
min,2000.000000,1.800000,2.300000,49.384000,0.000000,12.00000,10.000000,19.800000,8.000000,16.000000,0.010000,148.000000,0.080000,0.100000,0.100000,1.100000,0.000000,0.000000,39.400000
25%,2003.750000,8.200000,9.800000,108.346000,1.197500,78.00000,64.000000,23.200000,81.000000,81.000000,0.080000,1404.250000,2.070000,1.600000,1.600000,5.000000,0.000000,1.000000,62.675000
50%,2007.500000,19.800000,23.200000,164.422250,4.055000,89.00000,83.000000,25.500000,93.000000,93.000000,0.160000,4169.000000,7.925000,3.400000,3.400000,7.800000,0.000000,1.000000,71.300000
75%,2011.250000,47.725000,66.325000,248.099750,7.810000,96.00000,93.000000,26.400000,97.000000,97.000000,0.470000,12255.750000,23.850000,7.200000,7.300000,10.300000,0.000000,1.000000,75.400000
max,2015.000000,138.100000,224.900000,719.360500,17.870000,99.00000,99.000000,32.100000,99.000000,99.000000,21.680000,112418.000000,1379.860000,27.700000,28.600000,14.100000,1.000000,1.000000,83.800000


From the table we can see that the minimum life expectancy is 36 while the max is 89. The max adult mortality is 723 per thousand in population, while the min is 1. The average alcohol consumption per liters per capita is a lot lower than at I expected but the min is 0.01 which makes sense as some countries might have religious restrictions. It is also very disturbing seeing the low immunization coverage for some of the diseases. Especially when the average is pretty high for some. Overall this table gives us some interesting information on the distribution of the data. 

Below I am changing the column names to be lower case because they previously gave me issues the way it was originally formated. 

In [32]:
data.columns = data.columns.str.lower()

Now we are selecting the numerical and categorical columns so we can do some light data exploration.

In [33]:
numerical = data.select_dtypes(include=['float64', 'int64']).columns

In [34]:
categorical = data.select_dtypes(include=['object']).columns

Lets start by checking the differences between developed and developing countries.

In [35]:
pd.pivot_table(data, index='economy_status_developed', values=numerical)

ValueError: Grouper for 'economy_status_developed' not 1-dimensional

Above we can see that there are very big differences between the two types of countries. Developing countires have higher moratlity rates, and overall haelth issues. While the difference in immunization coverage for developed and developing countries is not a huge difference, developing countries still have less of it, which is important to note. Now lets take a look at the same data by each country.

In [ ]:
pd.pivot_table(data, index='country', values=numerical)

I realized that there are way too many countries to do some analysis on. I did not know that there was this many in the dataset. Regardless we can see that all these countries have distinct values. It would be interesting to see if we could possibly plot this. 

Now we are going to be ploting life expectancy against all other numerical columns to see if there is a relationship. We are using life expectancy because this is the column of interest that we are trying to predict later.

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 15))
axes = axes.flatten()
for i, col in enumerate(numerical):
    axes[i].scatter(data[col], data['life_expectancy'])  
    axes[i].set_title(f'{col} vs life_expectancy')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('life_expectancy')
plt.tight_layout()
plt.show()

From the graphs above there does not seem to be much of a relationship between life expectancy and the other columns in the dataset. By that I mean that there is no very noticable relationship.  The columns that seem to have some relationship are adult mortality (makes sense), infant deaths, percentage expenditure, under five deaths, hiv/aids, and income composition of resources. Of these adult mortality and income composition of resources seem to be the most coralated. It is also interesting to see infant deaths and under five deaths seem very similar. Same for the thiness columns.

Lastly there seems to be a group of populations that are constant in their life expectancy. For example if we look at income composition of resources we see that even at 0.0 the life expectancy changes up and down. This pattern can be seen through most graphs above. It might be some inconsistent or false reporting. 

Now we move on to graphing the distribution of the columns. 

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 15))  
axes = axes.flatten()
for i, col in enumerate(numerical):
    axes[i].hist(data[col])
    axes[i].set_title(col)
plt.tight_layout()
plt.show()

Everything seems alright, theres no big outliers. We will normalize and scale the data later. Lets now count the number of developed and devloping countries. 

In [ ]:
data['status'].value_counts() 

## Filling missing Values
Now that we have taken a look at our data, checked patterns, etc lets see what we can do about the missing values. First we must count how many values we are missing and where. 

In [ ]:
data.isnull().sum()

We have a lot of missing values all through out the columns. Most missing value is population, hepatitis and gdp. We can look into some patterns of which countries have missing values and try to see why. We can then start looking into countries that are similar to fill in the values. We'll start by seperating the data that has missing values.

In [ ]:
missing_data = data[data.isnull().any(axis=1)]

In [ ]:
missing_data.head(5)

In [ ]:
missing_data.country.value_counts()

The cell above tells us which countries seem to appear in the missing dataframe more often. We can use this to see if its a reocurring issue or a one time thing. If these countries appear often or if their values are missing often then we could look into it.

In [ ]:
missing_data.status.value_counts()

For now lets look at other interesting details. There seems to be more developing countires with missing values whihc makes sense because they might not have the infrustructure to keep count of all the data. 

Now we want to check the rows and see the ones that only have 1 missing value.

In [ ]:
missing_counts = missing_data.isnull().sum(axis=1)
df_one_missing = missing_data[missing_counts == 1]

In [ ]:
# missing_data[(missing_data['country'] == 'female') & (data['Pclass'] == 1) & (data['Fare'] >= 80) &

In [ ]:
len(df_one_missing)

In [ ]:
df_one_missing.isnull().sum()